In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

import pandas as pd

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-07-26 20:02:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-07-26 20:02:23 (8.92 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL_edu").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from GCS Buckets
from pyspark import SparkFiles
url ="https://storage.googleapis.com/team5k/health/education.csv"
spark.sparkContext.addFile(url)
df2 = spark.read.csv(SparkFiles.get("education.csv"), header=True, inferSchema=True)

# Show DataFrame
df2.show()

+---------+-----+---------------+-------------------------------+-------------------------+-------------------------------+-------------------------+-------------------------------------+------------------------------+------------------------------+-------------------------------------+------------------------------------------------------------+-------------------------------------------------------+-----------------------------------------------------------+----------------------------------------------------+-------------------------------------+------------------------------+------------------------------+-------------------------------------+------------------------------------------------------------+-------------------------------------------------------+-----------------------------------------------------------+----------------------------------------------------+-------------------------------------+------------------------------+----------------------------------------+-----

In [ ]:
#Check Shema
df2.printSchema()

root
 |-- FIPS Code: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Area name: string (nullable = true)
 |-- 2003 Rural-urban Continuum Code: double (nullable = true)
 |-- 2003 Urban Influence Code: double (nullable = true)
 |-- 2013 Rural-urban Continuum Code: double (nullable = true)
 |-- 2013 Urban Influence Code: double (nullable = true)
 |-- Less than a high school diploma, 1970: double (nullable = true)
 |-- High school diploma only, 1970: double (nullable = true)
 |-- Some college (1-3 years), 1970: double (nullable = true)
 |-- Four years of college or higher, 1970: double (nullable = true)
 |-- Percent of adults with less than a high school diploma, 1970: double (nullable = true)
 |-- Percent of adults with a high school diploma only, 1970: double (nullable = true)
 |-- Percent of adults completing some college (1-3 years), 1970: double (nullable = true)
 |-- Percent of adults completing college or higher, 1970: double (nullable = true)
 |-- Less than a hi

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://34.67.52.115/team5k"
config = {"user":"postgres", 
          "password": "team5kteam5k", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to RDS
df2.write.jdbc(url=jdbc_url, table="education", mode=mode, properties=config)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL_birth2").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from GCS Buckets
from pyspark import SparkFiles
url ="https://storage.googleapis.com/team5k/health/birth_date_rates.csv"
spark.sparkContext.addFile(url)
df3 = spark.read.csv(SparkFiles.get("birth_date_rates.csv"), header=True, inferSchema=True)

# Show DataFrame
df3.show()

+-----+--------------------+-----------+------+------+-----------------+
|Notes|              County|County Code|Deaths|Births|       Death Rate|
+-----+--------------------+-----------+------+------+-----------------+
| null|Jefferson County, AL|       1073|    84|  8706|             9.65|
| null|  Madison County, AL|       1089|    20|  4253|              4.7|
| null|   Mobile County, AL|       1097|    40|  5607|             7.13|
| null|Unidentified Coun...|       1999|   291| 40375|             7.21|
| null|Anchorage Borough...|       2020|    23|  4136|             5.56|
| null|Unidentified Coun...|       2999|    37|  6309|             5.86|
| null| Maricopa County, AZ|       4013|   311| 52633|             5.91|
| null|     Pima County, AZ|       4019|    41| 10983|             3.73|
| null|    Pinal County, AZ|       4021|    25|  4392|             5.69|
| null|Unidentified Coun...|       4999|    90| 13864|             6.49|
| null|  Pulaski County, AR|       5119|    47|  52

In [ ]:
# Write DataFrame to RDS
df3.write.jdbc(url=jdbc_url, table="birth_death_rate", mode=mode, properties=config)

In [ ]:
# Read in data from GCS Buckets
from pyspark import SparkFiles
url ="https://storage.googleapis.com/team5k/health/UnemploymentUrban6StateCounties2000-2019.csv"
spark.sparkContext.addFile(url)
df5 = spark.read.csv(SparkFiles.get("UnemploymentUrban6StateCounties2000-2019.csv"), header=True, inferSchema=True)

# Show DataFrame
df5.show()

+-------+-----+------------+--------------------+---------------------------+---------------+-----------------+----------------------+---------------------------+---------------+-----------------+----------------------+---------------------------+---------------+-----------------+----------------------+---------------------------+---------------+-----------------+----------------------+---------------------------+---------------+-----------------+----------------------+---------------------------+---------------+-----------------+----------------------+---------------------------+---------------+-----------------+----------------------+---------------------------+---------------+-----------------+----------------------+---------------------------+---------------+-----------------+----------------------+-------------------------+-------------+---------------+----------------------+---------------------------+---------------+-----------------+----------------------+----------------------

In [ ]:
# Write DataFrame to RDS
df5.write.jdbc(url=jdbc_url, table="unemployment", mode=mode, properties=config)